## 0. 기본 라이브러리, 함수 Import

In [0]:
%tensorflow_version 2.x

In [0]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import numpy as np
import tensorflow as tf

In [0]:
def vector_plot(vecs, 
                xlim, 
                ylim, 
                cols=['#1190FF', '#FF9A13'], 
                alpha=1):
  plt.rc_context({'axes.edgecolor':'orange', 'xtick.color':'red', 'ytick.color':'red'})
  plt.axvline(x=0, color='k', zorder=0) # x축
  plt.axhline(y=0, color='k', zorder=0) # y축

  for i in range(len(vecs)):
    if (isinstance(alpha, list)):
      alpha_i = alpha[i]
    else:
      alpha_i = alpha

    x = np.concatenate([[0,0], vecs[i]])
    plt.quiver([x[0]],
               [x[1]],
               [x[2]],
               [x[3]],
               angles='xy', scale_units='xy', scale=1, color=cols[i],
               alpha=alpha_i)
    plt.ylim(-xlim, xlim)
    plt.xlim(-ylim, ylim)
    plt.grid()


In [0]:
def plot_vector2d(vector2d, 
                  origin=[0, 0], 
                  **options):
  return plt.arrow(origin[0], origin[1], vector2d[0], vector2d[1],
                   head_width=0.2, head_length=0.3, length_includes_head=True,
                   **options)

In [0]:
def plot_transform(P_before, 
                   P_after, 
                   text_before, 
                   text_after, 
                   name, 
                   color=['#FF9A13', '#1190FF'], 
                   axis=[0,5,0,4],
                   arrows=False):
  if arrows:
    for vector_before, vector_after in zip(tf.transpose(P_before), tf.transpose(P_after)):
      plot_vector2d(vector_before, color='#FF9A13', linestyle='--')
      plot_vector2d(vector_after, color='#1190FF', linestyle='-')
    plt.rc_context({'axes.edgecolor':'orange', 'xtick.color':'red', 'ytick.color':'red'})
    plt.gca().add_artist(Polygon(tf.transpose(P_before), alpha=0.2))
    plt.gca().add_artist(Polygon(tf.transpose(P_after), alpha=0.3, color='#FF9A13'))
    plt.text(-.25, 1, text_before, size=18, color=color[1])
    plt.text(1.5, 0, test_after, size=18, color=color[0])
    plt.title(name, color='w')
    plt.axis(axis)
    plt.grid()

In [0]:
def evaluate(tensors):
  return tf.nest.pack_sequence_as(tensors, [t.numpy() if tf.is_tensor(t) else t for t in tf.next.flatten(tensors)])

## 1-1. Scalers, Vectors, Matrices and Tensors

In [0]:
import tensorflow as tf
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
# 3x3 랭크 2 텐서 만들기
rank_2_tensor_A = tf.ones([3, 3], name='MatrixA')
print('3x3 Rank 2 Tensor A: \n{}\n'.format(rank_2_tensor_A))

# 플로팅 데이터 타입을 가진 3x3 랭크 2 텐서 직접 만들기
rank_2_tensor_B = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]],
                              name='MatrixB',
                              dtype=tf.float32)
print('3x3 Rank 2 Tensor B: \n{}\n'.format(rank_2_tensor_B))

# 2개 텐서 더하기
rank_2_tensor_C = tf.add(rank_2_tensor_A, rank_2_tensor_B, name='MatrixC')
print('Rank 2 Tensor C with shape={} and elements: \n{}'.format(rank_2_tensor_C.shape,
                                                                rank_2_tensor_C))

3x3 Rank 2 Tensor A: 
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

3x3 Rank 2 Tensor B: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

Rank 2 Tensor C with shape=(3, 3) and elements: 
[[ 2.  3.  4.]
 [ 5.  6.  7.]
 [ 8.  9. 10.]]


In [10]:
# 차원이 같지 않을 때 생기는 일
two_by_three = tf.ones([2, 3])
try:
  incomapatible_tensor = tf.add(two_by_three, rank_2_tensor_B)
except:
  print("Incompatible shapes to add with two_by_tree of shape{0} and 3x3 Rank 2 \
Tensor B of shape {1}".format(two_by_three.shape, rank_2_tensor_B.shape))

Incompatible shapes to add with two_by_tree of shape(2, 3) and 3x3 Rank 2 Tensor B of shape (3, 3)


In [11]:
# 스칼라 a, c 행렬 B 만들기
rank_0_tensor_a = tf.constant(2, name='scaler_a', dtype=tf.float32)
rank_2_tensor_B = tf.constant([[1,2,3], [4,5,6], [7,8,9]], 
                              name='MatrixB',
                              dtype=tf.float32)
rank_0_tensor_c = tf.constant(3, name='scaler_c', dtype=tf.float32)

# a*B
multiply_scalar = tf.multiply(rank_0_tensor_a, rank_2_tensor_B)
# aB + c
rank_2_tensor_D = tf.add(multiply_scalar, rank_0_tensor_c, name='MatrixD')

print('Original Rank 2 Tensor B: \n{0} \n\nScaler a: {1} \
Rank 2 Tensor for aB: \n{2} \n\nScaler c: {3} \nRank 2 Tensor D = \
aB + c: \n{4}'.format(rank_2_tensor_B, 
                      rank_0_tensor_a, 
                      multiply_scalar,
                      rank_0_tensor_c, 
                      rank_2_tensor_D))

Original Rank 2 Tensor B: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]] 

Scaler a: 2.0 Rank 2 Tensor for aB: 
[[ 2.  4.  6.]
 [ 8. 10. 12.]
 [14. 16. 18.]] 

Scaler c: 3.0 
Rank 2 Tensor D = aB + c: 
[[ 5.  7.  9.]
 [11. 13. 15.]
 [17. 19. 21.]]


In [12]:
# E 행렬 만들기
rank_2_tensor_E = tf.constant([[1,2,3], [4,5,6]])
# E 행렬 전치
transpose_E = tf.transpose(rank_2_tensor_E, name='transposeE')

print('Rank 2 Tensor E of shape: {0} and elements: \n{1}\n\
Transpose of Rank 2 Tensor E of shape: {2} \
and elements: \n{3}'.format(rank_2_tensor_E.shape,
                            rank_2_tensor_E,
                            transpose_E.shape,
                            transpose_E))

Rank 2 Tensor E of shape: (2, 3) and elements: 
[[1 2 3]
 [4 5 6]]
Transpose of Rank 2 Tensor E of shape: (3, 2) and elements: 
[[1 4]
 [2 5]
 [3 6]]


In [13]:
# 벡터 b 만들기
rank_1_tensor_b = tf.constant([[4.], [5.], [6.]])
# 벡터 b를 행렬 A에 브로드캐스팅하기, F = A + b 형태
rank_2_tensor_F = tf.add(rank_2_tensor_A, rank_1_tensor_b, name='broadcaseF')

print('Rank 2 Tensor A: \n{0}\n \nRank 1 Tensor b: \n{1} \
\nRank 2 tensor F = A + b:\n{2}'.format(rank_2_tensor_A,
                                        rank_1_tensor_b,
                                        rank_2_tensor_F))

Rank 2 Tensor A: 
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
 
Rank 1 Tensor b: 
[[4.]
 [5.]
 [6.]] 
Rank 2 tensor F = A + b:
[[5. 5. 5.]
 [6. 6. 6.]
 [7. 7. 7.]]


## 1-2. Multiplying Matrices and Vectors

In [14]:
# (2, 3) A 행렬, (3, 4) B 행렬
mmv_matrix_A = tf.ones([2, 3], name='matrix_A')
mmv_matrix_B = tf.constant([[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]],
                           name='matrix_B',
                           dtype=tf.float32)

# 행렬 곱셈(내적), C = AB C는 (2, 3) 행렬
matrix_multiply_C = tf.matmul(mmv_matrix_A, mmv_matrix_B, name='matrix_multiply_C')

print('Matrix A: shape {0} \nelements: \n{1} \n\nMatrix B: shape {2} \
\belements: \n{3}\nMatrix C: shape {4} \
\nelements: \n{5}'.format(mmv_matrix_A.shape,
                          mmv_matrix_A,
                          mmv_matrix_B.shape,
                          mmv_matrix_B,
                          matrix_multiply_C.shape,
                          matrix_multiply_C))

Matrix A: shape (2, 3) 
elements: 
[[1. 1. 1.]
 [1. 1. 1.]] 

Matrix B: shape (3, 4) elements: 
[[1. 2. 3. 4.]
 [1. 2. 3. 4.]
 [1. 2. 3. 4.]]
Matrix C: shape (2, 4) 
elements: 
[[ 3.  6.  9. 12.]
 [ 3.  6.  9. 12.]]


In [15]:
# multiply는 개별 요소의 곱 matmul은 행렬곱
# (3, 3) 행렬 A, B를 만든다
element_matrix_A = tf.ones([3, 3], name='element_matrix_A')
element_matrix_B = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]],
                                name='element_matrix_B',
                                dtype=tf.float32)

# 행렬 A, B에 대해 개별 요소의 곱
element_wise_C = tf.multiply(element_matrix_A, 
                             element_matrix_B, 
                             name='element_wise_C')

print('Matrix A: shape {0} \nelements: \n{1} \n\nMatrix A: shape {2} \nelements: \n{3}\n\
Matrix C: shape {4} \nelements: \n{5}'.format(element_matrix_A.shape,
                                              element_matrix_A,
                                              element_matrix_B.shape,
                                              element_matrix_B,
                                              element_wise_C.shape,
                                              element_wise_C))

Matrix A: shape (3, 3) 
elements: 
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]] 

Matrix A: shape (3, 3) 
elements: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
Matrix C: shape (3, 3) 
elements: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


In [16]:
masking = tf.constant([[1,0,0], [0,1,0], [0,0,1]], dtype=tf.float32)
masking_result = tf.multiply(element_matrix_B, masking)
print(f"masking result: {masking_result}")

masking result: [[1. 0. 0.]
 [0. 5. 0.]
 [0. 0. 9.]]


In [17]:
# (3, 3) 행렬 A, B 만들기
dot_matrix_A = tf.ones([3, 3], name='dot_matrix_A')
dot_matrix_B = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]],
                                name='dot_matrix_B',
                                dtype=tf.float32)

# 내적 A, B
dot_product_C = tf.tensordot(dot_matrix_A, 
                             dot_matrix_B, 
                             axes=1,
                             name='dot_product_C')

print('Matrix A: shape {0} \nelements: \n{1} \n\nMatrix B: \
shape {2} \nelements: \n{3}\n Matrix C: shape {4}\
\nelements: \n{5}'.format(dot_matrix_A.shape,
                          dot_matrix_A,
                          dot_matrix_B.shape,
                          dot_matrix_B,
                          dot_product_C.shape,
                          dot_product_C))

Matrix A: shape (3, 3) 
elements: 
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]] 

Matrix B: shape (3, 3) 
elements: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
 Matrix C: shape (3, 3)
elements: 
[[12. 15. 18.]
 [12. 15. 18.]
 [12. 15. 18.]]


In [0]:
# 행렬의 곱셈 분배 법칙을 확인하기 위한 행렬 생성
matrix_A = tf.constant([[1, 2], [3, 4]], name='matrix_a')
matrix_B = tf.constant([[5, 6], [7, 8]], name='matrix_b')
matrix_C = tf.constant([[9, 1], [2, 3]], name='matrix_c')

In [19]:
print('Matrix A: \n{} \n\nMatrix B: \n{} \n\nMatrix C: \n{}\n'.format(matrix_A,
                                                                      matrix_B,
                                                                      matrix_C))

# AB + AC
distributive_RHS = tf.add(tf.matmul(matrix_A, matrix_B), 
                          tf.matmul(matrix_A, matrix_C),
                          name='RHS')

# A(B+C)
distributive_LHS = tf.matmul(matrix_A, 
                             (tf.add(matrix_B, matrix_C)),
                             name='LHS')

predictor = tf.reduce_all(tf.equal(distributive_RHS, distributive_LHS))

def true_print(): print('Distributive property is valid \
RHS: AB + AC: \n{} \n\nLHS: A(B+C): \n{}'.format(distributive_RHS,
                                                 distributive_LHS))

def false_print(): print('You Broke the Distributive Property of Matrix\
RHS: AB + AC: \n{} \n\nis NOT Equal to LHS: A(B+C): \n{}\
'.format(distributive_RHS,
         distributive_LHS))

tf.cond(predictor, true_print, false_print)

Matrix A: 
[[1 2]
 [3 4]] 

Matrix B: 
[[5 6]
 [7 8]] 

Matrix C: 
[[9 1]
 [2 3]]

Distributive property is valid RHS: AB + AC: 
[[32 29]
 [78 65]] 

LHS: A(B+C): 
[[32 29]
 [78 65]]


In [20]:
# 결합 법칙
print('Matrix A: \n{} \n\nMatrix B: \n{} \n\nMatrix C: \n{}\n'.format(matrix_A,
                                                                      matrix_B,
                                                                      matrix_C))

#(AB)C
associative_RHS = tf.matmul(tf.matmul(matrix_A, matrix_B), matrix_C)

#A(BC)
associative_LHS = tf.matmul(matrix_A, tf.matmul(matrix_B, matrix_C))

predictor = tf.reduce_all(tf.equal(associative_RHS, associative_LHS))

def true_print(): print('Associatie property is valid \
RHS: (AB)C: \n{} \n\nLHS: A(BC): \n{}'.format(associative_RHS,
                                              associative_LHS))

def false_print(): print('You Broke the Associative Property of Matrix \
RHS: (AB)C: \n{} \n\nLHS: A(BC): \n{}'.format(associative_RHS,
                                              associative_LHS))

tf.cond(predictor, true_print, false_print)

Matrix A: 
[[1 2]
 [3 4]] 

Matrix B: 
[[5 6]
 [7 8]] 

Matrix C: 
[[9 1]
 [2 3]]

Associatie property is valid RHS: (AB)C: 
[[215  85]
 [487 193]] 

LHS: A(BC): 
[[215  85]
 [487 193]]


In [21]:
 # 행렬곱의 교환 법칙은 성립하지 않는다.
 print('Matrix A: \n{} \n\nMatrix B: \n{}\n'.format(matrix_A, matrix_B))

 # Matrix A times B
 commutative_RHS = tf.matmul(matrix_A, matrix_B)

 # Matrix B times A
 commutative_LHS = tf.matmul(matrix_B, matrix_A)

 predictor = tf.logical_not(tf.reduce_all(tf.equal(commutative_RHS, 
                                                   commutative_LHS)))
 
 def true_print(): print('Matrix Multiplication is not commutative \
 RHS: (AB): \n{} \n\nLHS: (BA): \n{}'.format(commutative_RHS,
                                             commutative_LHS))
 
 def false_print(): print('You made Matrix Multiplication commutative \
 RHS: (AB): \n{} \n\nLHS: (BA): \n{}'.format(commutative_RHS,
                                             commutative_LHS))
 
 tf.cond(predictor, true_print, false_print)

Matrix A: 
[[1 2]
 [3 4]] 

Matrix B: 
[[5 6]
 [7 8]]

Matrix Multiplication is not commutative RHS: (AB): 
[[19 22]
 [43 50]] 

LHS: (BA): 
[[23 34]
 [31 46]]


In [23]:
# 행렬 곱을 전치해도 같다.
print('Matrix A: \n{} \n\nMatrix B: \n{}\n'.format(matrix_A, matrix_B))

# Tensorflow의 전치 함수
transpose_RHS = tf.transpose(tf.matmul(matrix_A, matrix_B))

# tf.matmul 함수에 전치 파라미터를 적용하면, 전치 후에 행렬곱을 수행한다.
transpose_LHS = tf.matmul(matrix_B, matrix_A, transpose_a=True, transpose_b=True)

predictor = tf.reduce_all(tf.equal(transpose_RHS, transpose_LHS))
def true_print(): print('Transpose property is valid \
RHS: (AB):^T \n{} \n\nLHS: (B^T A^T): \n{}'.format(transpose_RHS,
                                                   transpose_LHS))

def false_print(): print('You Broke the Transpose Property of Matrix \
RHS: (AB):^T \n{} \n\nLHS: (B^T A^T): \n{}'.format(transpose_RHS,
                                                   transpose_LHS))

tf.cond(predictor, true_print, false_print)

Matrix A: 
[[1 2]
 [3 4]] 

Matrix B: 
[[5 6]
 [7 8]]

Transpose property is valid RHS: (AB):^T 
[[19 43]
 [22 50]] 

LHS: (B^T A^T): 
[[19 43]
 [22 50]]


## 1-3. Identity ans Inverse Matrices

In [24]:
# 단위 행렬을 생성한다.
identity_matrix_I = tf.eye(3, 3, dtype=tf.float32, name='IdentityMatrixI')
print('Identity matrix I: \n{}\n'.format(identity_matrix_I))

# 3x1 벡터 x를 만든다.
iim_vector_x = tf.constant([[4], [5], [6]], name='Vector_x', dtype=tf.float32)
print('Vector x: \n{}\n'.format(iim_vector_x))

# Ix 행렬곱 연산은 x를 반환한다.
iim_matrix_C = tf.matmul(identity_matrix_I, iim_vector_x, name='MatrixC')
print('Matrix C from Ix: \n{}'.format(iim_matrix_C))

Identity matrix I: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Vector x: 
[[4.]
 [5.]
 [6.]]

Matrix C from Ix: 
[[4.]
 [5.]
 [6.]]


In [25]:
iim_matrix_A = tf.constant([[2, 3], [2, 2]], name='MatrixA', dtype=tf.float32)

try:
  # Tensorflow 역행렬 함수
  inverse_matrix_A = tf.round(tf.linalg.inv(iim_matrix_A) * 100) / 100

  # tf.eye 함수를 사용해서 단위 행렬 생성
  identity_matrix = tf.eye(2, 2, dtype=tf.float32, name='identity')

  iim_RHS = identity_matrix
  iim_LHS = tf.matmul(inverse_matrix_A, iim_matrix_A, name='LHS')

  predictor = tf.reduce_all(tf.equal(iim_RHS, iim_LHS))
  def true_print(): print('A^-1 times A equals the Identity Matrix \
Matrix A: \n{0} \n\nInverse of Matrix A: \n{1} \n\nRHS: I: \n{2} \
\n\nLHS: A^(-1) A: \n{3}'.format(iim_matrix_A,
                                 inverse_matrix_A,
                                 iim_RHS,
                                 iim_LHS))
  def false_print(): print('Condition Failed')
  tf.cond(predictor, true_print, false_print)

except:
  print('A^-1 doesnt exist \
Matrix A: \n{} \n\nInverse of Matrix A: \n{} \n\nRHS: I: \n{} \n\
LHS: (A^(-1) A): \n{}'.format(iim_matrix_A,
                              inverse_matrix_A,
                              iim_RHS,
                              iim_LHS))

A^-1 times A equals the Identity Matrix Matrix A: 
[[2. 3.]
 [2. 2.]] 

Inverse of Matrix A: 
[[-1.   1.5]
 [ 1.  -1. ]] 

RHS: I: 
[[1. 0.]
 [0. 1.]] 

LHS: A^(-1) A: 
[[1. 0.]
 [0. 1.]]


In [26]:
sys_matrix_A = tf.constant([[2, 3], [4, 9]], dtype=tf.float32)
sys_vector_B = tf.constant([[6], [15]], dtype=tf.float32)
print('Matrix A: \n{} \n\nVector B: \n{}\n'.format(sys_matrix_A,
                                                   sys_vector_B))

sys_x = tf.matmul(tf.linalg.inv(sys_matrix_A), sys_vector_B)
print('Vector x is: \n{} \nWhere x = {} and y = {}'.format(sys_x,
                                                           sys_x[0],
                                                           sys_x[1]))

Matrix A: 
[[2. 3.]
 [4. 9.]] 

Vector B: 
[[ 6.]
 [15.]]

Vector x is: 
[[1.5      ]
 [1.0000001]] 
Where x = [1.5] and y = [1.0000001]
